# Coursera Reviews EDA
- Explore the course metadata and explore relationships between features with rating as the target
- Identify potentially interesting features to pair with sentiment
- Explore word distributions, keywords, etc in reviews.

In [1]:
!pip -q install datasets transformers huggingface_hub langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import scipy.stats as stats
import re
import nltk
from nltk.corpus import words
from collections import Counter
import math
import unicodedata
import time
from tqdm import tqdm
import pickle
from langdetect import detect
import warnings

In [ ]:
nltk.download('words')

In [ ]:
courses_df = pd.read_csv('/kaggle/input/course-reviews-on-coursera/Coursera_courses.csv')
courses_df.head(3)

In [ ]:
reviews_df = pd.read_csv('/kaggle/input/course-reviews-on-coursera/Coursera_reviews.csv')
reviews_df.head(3)

In [ ]:
reviews_df.reviews

In [ ]:
plt.figure(figsize=(8,6))

counts, bins, patches = plt.hist(reviews_df['rating'], 
                                 bins=[0.5, 1.5, 2.5, 3.5, 4.5, 5.5],
                                 edgecolor='black')
    
plt.xticks([1, 2, 3, 4, 5])

for count, x in zip(counts, [1, 2, 3, 4, 5]):
    plt.text(x, count, f'{int(count)/len(reviews_df):.1%}', ha='center', va='bottom')
    
plt.xlabel('Course Rating')
plt.ylabel('Count')
plt.title('Distribution of Coursera Course Ratings')
plt.grid(True, alpha=0.3)

# Ensure y-axis starts at 0 and has some padding on top
plt.margins(y=0.1)
plt.tight_layout()
plt.savefig('rating_hist.png')
# Display the plot
plt.show()

In [ ]:
def plot_monthly_avg(df, dates, ratings):
    # Plot average ratings over time
    reviews_df[dates] = pd.to_datetime(reviews_df[dates])
    
    reviews_df['year_month'] = reviews_df[dates].dt.to_period('M')
    monthly_avg = reviews_df.groupby('year_month')[ratings].mean().reset_index()
    monthly_avg['year_month'] = monthly_avg['year_month'].dt.to_timestamp()
    
    # Plot the monthly average
    plt.figure(figsize=(12, 6))
    plt.plot(monthly_avg['year_month'], monthly_avg['rating'], 
             marker='o', linestyle='-', linewidth=1.5)
    
    # Plot overall average
    plt.hlines(y=monthly_avg[ratings].mean(), 
               xmin=monthly_avg['year_month'].min(), 
               xmax=monthly_avg['year_month'].max(), 
               colors='g', linestyles='dotted', label='Avg. Rating')
    
    plt.xlabel('Date')
    plt.ylabel('Average Course Rating')
    plt.title('Average Ratings Over Time (Monthly)')
    plt.grid(True, alpha=0.3)
    
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)
    plt.legend()
    # Adjust layout to prevent label cutoff
    plt.tight_layout()
    plt.savefig('rating_over_time.png')
    # Show the plot
    plt.show()

plot_monthly_avg(reviews_df, 'date_reviews', 'rating')

In [ ]:
def plot_top_categories(df, column_name, n=20):

    uni_counts = df[column_name].value_counts().head(n)
    
    # Create horizontal bar chart
    plt.figure(figsize=(10, 8))
    uni_counts.plot(kind='barh')
    
    plt.xlabel('Number of Occurrences')
    plt.ylabel('Universities')
    plt.title(f'Top {n} Most Frequent Universities')
    plt.grid(True, alpha=0.3, axis='x')
    
    # Invert y-axis to have highest count at top
    plt.gca().invert_yaxis()
    
    # Add counts on the bars
    for i, v in enumerate(uni_counts):
        plt.text(v, i, str(v), va='center')
    
    # Adjust layout
    plt.tight_layout()
    plt.savefig('top_universities.png')
    plt.show()

plot_top_categories(courses_df, 'institution')

#### Merge the dataframes

In [ ]:
merged_df = pd.merge(courses_df, reviews_df, 
                    on='course_id', 
                    how='inner')  # 'inner' keeps only matching records

In [ ]:
# Calculate average ratings per university and counts for each
avg_ratings = merged_df.groupby('institution')['rating'].mean()
rating_counts = merged_df.groupby('institution')['rating'].count()

uni_stats = pd.DataFrame({
    'avg_rating': avg_ratings,
    'n_ratings': rating_counts
})

# Find outliers with IQR
Q1 = uni_stats['avg_rating'].quantile(0.25)
Q3 = uni_stats['avg_rating'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = uni_stats[(uni_stats['avg_rating'] < lower_bound) | 
                    (uni_stats['avg_rating'] > upper_bound)].reset_index()

def get_hi_low(row, hi, low):
    if row['avg_rating'] > hi:
        return 'High'
    elif row['avg_rating'] < low:
        return 'Low'
    else:
        return "Normal"

outliers['hi_lo'] = outliers.apply(lambda row: get_hi_low(row, upper_bound, lower_bound), axis=1)

print("Potential Outliers (using IQR method):")
if not outliers.empty:
    #print(outliers)
    # Construct Markdown table manually
    md_table = f"| {'Institution'} | Average Rating | Number of Ratings |\n"
    md_table += "|------------|----------------|-------------------|\n"
    for _, row in outliers.iterrows():
        md_table += f"| {row['institution']} | {row['avg_rating']:.2f} | {row['n_ratings']} | {row['hi_lo']} |\n"
    md_table += f"| **Overall Avg** | **{uni_stats['avg_rating'].mean():.2f}** | **{len(merged_df)}** | N/A |\n"
    print(md_table)
else:
    print("No outliers detected.")

Potential Average Rating Outliers (using IQR method):
| Institution | Average Rating | Number of Ratings | High/Low |
|------------|----------------|-------------------| ------- |
| Advancing Women in Product | 4.33 | 9 | Low |
| ESCP Business School | 4.31 | 153 | Low |
| IE Business School | 4.01 | 82 | Low |
| LearnQuest | 4.32 | 246 | Low |
| New York Institute of Finance | 3.40 | 368 | Low |
| Novosibirsk State University  | 4.03 | 195 | Low |
| Saint Petersburg State University | 3.33 | 204 | Low |
| University of New Mexico | 1.00 | 6 | Low |
| Yandex | 3.42 | 290 | Low |
| **Overall Avg** | **4.65** | **1454711** | N/A |

#### Cleaning the Reviews - Word Counts + Entropy

In [ ]:
# Find entries with no review
print(f"There are {sum(merged_df.reviews.isna())} entries with no review text")

In [ ]:
df = merged_df.dropna(subset=['reviews'])
print(f"Dropped {len(merged_df) - len(df)} entries with no reviews")

In [ ]:
#df = df[df['reviews'].str.len() < 10]

print(f"There are {len(df[df['reviews'].str.len() < 5])} entries with < 5 characters")
print(f"There are {len(df[df['reviews'].str.len() < 10])} entries with < 10 characters")

We notice that a lot of the remaining non null entries have very short reviews. Let's look at some of the entries where there are less than 4 characters. I picked 4 since the word "good" comes up quite frequently.

In [ ]:
df[df['reviews'].str.len() < 4].head(10).reviews

If we limit it to 10 characters, we see reviews like "its good" or "good" so this seems like a reasonable limit. There may still be entries with nonsensical words.

In [ ]:
df[df['reviews'].str.len() < 10].head(8).reviews

In [ ]:
# Review length (word count)
word_count_df = df.copy()
word_count_df['word_count'] = word_count_df['reviews'].apply(lambda x: len(str(x).split()))
print("\nReview Word Count Stats:")
print(df['word_count'].describe().to_markdown())

plt.figure(figsize=(10, 6))
sns.histplot(word_count_df['word_count'], bins=10)
plt.title('Distribution of Review Word Counts')
plt.xlabel('Word Count')
plt.ylabel('Frequency')
plt.show()

This plot is not very meaningful, need to cut the outliers to get better idea

In [ ]:
def plot_word_count_distribution(original_df, review_col='reviews', percentile_cutoff=95, sample_size=5):

    df = original_df.copy()
    df['word_count'] = df[review_col].apply(lambda x: len(str(x).split()))

    # Set word count cutoff to a given percentile. Defult to 95
    cutoff = np.percentile(df['word_count'], percentile_cutoff)
    print(f"\n{percentile_cutoff}th Percentile Cutoff: {cutoff:.0f} words")

    main_range = df['word_count'][df['word_count'] <= cutoff]
    outliers = df[df['word_count'] > cutoff]
    
    # Plot histogram for main range
    plt.figure(figsize=(10, 6))
    sns.histplot(main_range, bins=30)
    plt.title(f'Distribution of Word Counts (Up to {cutoff:.0f} Words)')
    plt.xlabel('Word Count')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('word_count_hist.png')
    plt.show()

    # Make output df of outliers and print to markdown
    if not outliers.empty:
        
        outlier_sample = outliers[[review_col, 'word_count']].head(sample_size)
        
        md_table = "| Review Text | Word Count |\n"
        md_table += "|-------------|------------|\n"
        for _, row in outlier_sample.iterrows():
            # Truncate long text for display (optional)
            text = row[review_col] if len(row[review_col]) < 50 else row[review_col][:47] + "..."
            md_table += f"| {text} | {row['word_count']} |\n"
        
        print("\nSample of Outlier Reviews:")
        print(md_table)


plot_word_count_distribution(df, percentile_cutoff=97.5)

Sample of Outlier Reviews:
| Review Text | Word Count |
|-------------|------------|
| This is an extremely basic course. Machine lear... | 152 |
| I just started week 3 , I have to admit that It... | 93 |
| This course is absolute garbage.  You get no fe... | 107 |
| Have to give a star so I will give it one.  Oth... | 131 |
| I would rename this course as Programming Octav... | 125 |

Next, we'll attempt to define a meaningful review as one with low entropy. Higher entropy suggests a more random distribution of characters, which is more likely to be gibberish over structured language.

**NOTE**: We'll use unicodedata.category(c).startswith('L') to make sure the words have any unicode characters not just those in english alphabet. Additionally, chinese apparently can often have higher entropy, so we need to check if a review is in chinese and add seprate conditions. The same applies for cyrilic apparently.



In [ ]:
def calculate_entropy(text):
    """Calculate Shannon entropy of the text to detect randomness."""
    if not text:
        return 0
    if not isinstance(text, str) or pd.isna(text):
        return 0  # Return 0 for NaN or non-string values
    text = str(text).lower()
    length = len(text)
    if length == 0:  # Handle empty strings
        return 0
    char_counts = Counter(text)
    entropy = -sum((count/length) * math.log2(count/length) for count in char_counts.values())
    return entropy

# Check to see if text has chinese or cyrilic since they can have higher entropy
def has_chinese(text):
    """Check if text contains Chinese characters."""
    return any('\u4e00' <= char <= '\u9fff' for char in text)

def has_cyrillic(text):
    """Check if text contains Cyrillic characters."""
    return any('\u0400' <= char <= '\u04ff' for char in text)

In [ ]:
sns.histplot(merged_df['reviews'].apply(calculate_entropy)

In [ ]:
def plot_entropy_distribution(test_df, review_col='reviews', 
                             base_entropy_english=4.5, entropy_scale=0.5,
                             max_entropy_chinese=7.5, max_entropy_other=5.5):
    df = test_df.copy()

    
    # Calculate entropies and lengths
    df['entropy'] = df[review_col].apply(calculate_entropy)
    df['length'] = df[review_col].apply(lambda x: len(str(x)))
    
    # Separate by language for analysis
    df['is_chinese'] = df[review_col].apply(lambda x: any('\u4e00' <= c <= '\u9fff' for c in str(x)))
    df['is_cyrillic'] = df[review_col].apply(lambda x: any('\u0400' <= c <= '\u04ff' for c in str(x)))
    
    # Compute dynamic English threshold for mean length (for reference)
    mean_length = df['length'].mean()
    max_entropy_english_mean = base_entropy_english + (mean_length / 100) * entropy_scale
    
    # Stats
    print("Entropy Statistics:")
    print(df['entropy'].describe())
    print(f"\nMean Review Length: {mean_length:.0f} characters")
    print(f"Scaled max_entropy_english at mean length: {max_entropy_english_mean:.2f}")
    
    # Plot histogram
    plt.figure(figsize=(12, 6))
    sns.histplot(df['entropy'], bins=50, color='blue', label='Entropy Distribution')
    
    # Add threshold lines
    plt.axvline(x=base_entropy_english, color='red', linestyle='--', 
                label=f'Base English Threshold ({base_entropy_english})')
    plt.axvline(x=max_entropy_english_mean, color='orange', linestyle='--', 
                label=f'Scaled English at Mean Length ({max_entropy_english_mean:.2f})')
    plt.axvline(x=max_entropy_other, color='green', linestyle='--', 
                label=f'Cyrillic/Other Threshold ({max_entropy_other})')
    plt.axvline(x=max_entropy_chinese, color='purple', linestyle='--', 
                label=f'Chinese Threshold ({max_entropy_chinese})')
    
    # Customize plot
    plt.title('Distribution of Review Entropies')
    plt.xlabel('Entropy')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('review_entropy_dist.png')
    plt.show()
    
    # Optional: Separate by language
    plt.figure(figsize=(12, 6))
    sns.histplot(df[df['is_chinese']]['entropy'], bins=30, kde=True, color='purple', label='Chinese')
    sns.histplot(df[df['is_cyrillic']]['entropy'], bins=30, kde=True, color='green', label='Cyrillic')
    sns.histplot(df[~df['is_chinese'] & ~df['is_cyrillic']]['entropy'], bins=30, 
                 color='blue', label='English/Other')
    plt.title('Entropy Distribution by Language')
    plt.xlabel('Entropy')
    plt.ylabel('Frequency')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('review_entropy_lang.png')
    plt.show()


plot_entropy_distribution(merged_df.copy())

In [ ]:
def is_meaningful_review_multilingual(text, min_alpha_ratio=0.2, 
                                    max_entropy_english=4.5, 
                                    max_entropy_chinese=7.5, 
                                    max_entropy_other=5.5):
    # Get rid of empty reviews
    if not text or pd.isna(text):
        return False
    
    text = str(text).lower()

    # Count Unicode letters for alpha ratio
    alpha_chars = sum(unicodedata.category(c).startswith('L') for c in text)
    total_chars = len(text.replace(" ", ""))
    
    if total_chars == 0 or alpha_chars / total_chars < min_alpha_ratio:
        return False
        
    entropy = calculate_entropy(text)
    
    # Chinese reviews
    if has_chinese(text):
        return total_chars >= 5 and entropy <= max_entropy_chinese
    
    # Cyrillic (e.g., Russian) reviews
    if has_cyrillic(text):
        return total_chars >= 5 and entropy <= max_entropy_other
    
    # English or mixed reviews
    if entropy > max_entropy_english:
        return False
    
    # Make sure entropy is smaller than max_entropy param
    entropy = calculate_entropy(text)
    if entropy > max_entropy_english:  # High entropy indicates randomness
        return 0
    
    # Make sure there is at least one real word in each entry (this will take a long time)
    #word_list = set(words.words())  # NLTK's English word list
    #tokens = re.findall(r'\b[a-z]+\b', text)  # return list of real words in review entry
    #if not tokens:  # No words found
    #    return 0
    
    #dict_words = sum(1 for token in tokens if token in word_list)
    #dict_ratio = dict_words / len(tokens)
    #if dict_ratio < min_dict_ratio:
    #    return 0
    
    return 1

def flag_meaningless_reviews(test_df, review_col='reviews'):
    """Apply the meaningfulness check to a dataframe column and add a flag."""
    df = test_df.copy()
    df['is_meaningful'] = df[review_col].apply(is_meaningful_review)
    return df

# DF with column for meaningless/meaningful tag
result_df = flag_meaningless_reviews(merged_df.copy())

# DF filtered for only tag 1
meaningful_df = result_df[result_df['is_meaningful'] == 1].copy()

# Summary of meaningful vs. meaningless
print("\nSummary:")
print(result_df['is_meaningful'].value_counts())


What do some of the meaningless phrases look like? We see below that the alphabetic ratio and entropy are hyperparameters that need tuning. Some legitimate cases of "meaningless" are:
1. The entry "NaN'
2. A single punctuation mark or typed out emoji

**NEED TO ADDRESS**
- Right now, it's detecting chinese characters as "meaningless" so wee nee to address this.

In [ ]:
print(result_df[result_df['is_meaningful'] == False].sample(5).reviews.values)

Below is an example of a substantive review written in chinese characters that has a higher entropy than it would in english.

In [ ]:
chinese_string = '这是第一门自己申请助学金的课程，而霍普金斯大学也是我非常敬仰的学校，然而这门课程却不如我预想中的完美，第一是视频上来看，图形资源比较匮乏，多媒体资源没有很好的和讲课内容匹配，比如课程音频说的here 在屏幕上并没有显示出来具体的指示标识；第二是相关的资源太少了，如果相比莱顿大学的腹部解剖那门课程来说的确是比较枯燥。不过因为自己本身是这个专业的所以看起来障碍不是太大，不过对于没有肿瘤学基础的人来说上面的问题可能影响会比较大一些，希望团队能在这一方面继续改进。最后还是要感谢各位老师带来这么一门优秀的课程，我从中学到了很多！再次感谢'
alphs = sum(unicodedata.category(c).startswith('L') for c in chinese_string)
total = len(chinese_string.replace(" ", ""))
entropy = calculate_entropy(chinese_string)
print(f'Sum of chars = {alphs}')
print(f"Total Chars = {total}")
print(f"Ratio = {alphs/total}")
print(f"Entropy = {entropy}")

In [ ]:
# Below is the english translation of the above chinese text
translated_string = "This is the first course for which I applied for a scholarship myself, and Johns Hopkins University is a school that I admire very much. However, this course is not as perfect as I expected. First, judging from the video, the graphic resources are relatively scarce, and the multimedia resources do not match the lecture content very well. For example, the course audio says here but there is no specific indicator on the screen; second, there are too few related resources. Compared with the abdominal anatomy course at Leiden University, it is indeed quite boring. However, since I am a major in this field, the obstacles do not seem to be too big. However, for those who do not have a foundation in oncology, the above problems may have a greater impact. I hope the team can continue to improve in this regard. Finally, I would like to thank all the teachers for bringing such an excellent course. I have learned a lot from it! Thanks again"
alphs = sum(unicodedata.category(c).startswith('L') for c in translated_string)
total = len(translated_string.replace(" ", ""))
entropy = calculate_entropy(translated_string)
print(f'Sum of chars = {alphs}')
print(f"Total Chars = {total}")
print(f"Ratio = {alphs/total}")
print(f"Entropy = {entropy}")

In [ ]:

result_df[result_df['is_meaningful'] == 0].iloc[67477].reviews

#### Distribution of Reviews by Language
- Here we check to see how many reviews are in different languages. Let's investigate if there is a statistically significant difference in average entropy between different langugages' reviews
- **NOTE:** Tagging the language of the DF takes ~2 hours on the Kaggle CPU

In [ ]:
from langdetect import detect
import warnings

# Suppress langdetect warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

def detect_language(text):

    if not isinstance(text, str) or pd.isna(text) or len(text.strip()) < 3:
        return 'unknown'  # For NaN, empty, or very short text
    try:
        return detect(text)
    except:
        return 'unknown'  # Fallback for any detection errors

# Get distribution of langugages using langdetect
def plot_language_distribution(df, review_col='reviews'):

    # Detect language for each review
    tqdm.pandas(desc="Detecting Languages") 
    df['language'] = df[review_col].progress_apply(detect_language)
    # Save this df with tagged languages to file
    df.to_pickle('merged_df_w_langs.pkl')
    # Count reviews per language
    language_counts = df['language'].value_counts().reset_index()
    language_counts.columns = ['language', 'count']
    
    # Print stats
    print("Language Distribution:")
    print(language_counts.to_markdown())
    
    # Plot bar chart
    plt.figure(figsize=(12, 6))
    sns.barplot(data=language_counts, x='language', y='count')
    plt.title('Number of Reviews by Language')
    plt.xlabel('Language (ISO 639-1 Code)')
    plt.ylabel('Number of Reviews')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, alpha=0.3)
    
    # Add count labels on top of bars
    for i, row in language_counts.iterrows():
        plt.text(i, row['count'], row['count'], ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('num_reviews_language.png')
    plt.show()
    return df # Return back the original df WITH a new language column

# Install langdetect if needed: pip install langdetect
lang_df = plot_language_distribution(merged_df.copy())

Detecting Languages:   2%|▏         | 31524/1454711 [02:23<1:31:55, 258.04it/s]

In [ ]:
# FUnction to statistically detect differences in entropy of languages
def test_entropy_differences(df, review_col='review_comment'):
    """Test for significant differences in entropy across languages.
       Feed this a df with entropies calculated/languages tagged for faster runtime"""
    # Check if entropy/language tags are there
    if 'entropy' not in df.columns:
        df['entropy'] = df[review_col].apply(calculate_entropy)
    if 'language' not in df.columns:
        df['language'] = df[review_col].apply(detect_language)
    
    # Filter out 'unknown' and languages with too few samples (< 2)
    language_counts = df['language'].value_counts()
    valid_languages = language_counts[language_counts >= 2].index
    df_filtered = df[df['language'].isin(valid_languages)].copy()
    
    if len(valid_languages) < 2:
        print("Not enough languages with sufficient samples (>= 2 reviews) for comparison.")
        return
    
    # Group entropies by language
    grouped_entropies = [df_filtered[df_filtered['language'] == lang]['entropy'].values 
                         for lang in valid_languages]

        md_table = "| Review Text | Word Count |\n"
        md_table += "|-------------|------------|\n"
        for _, row in outlier_sample.iterrows():
            # Truncate long text for display (optional)
            text = row[review_col] if len(row[review_col]) < 50 else row[review_col][:47] + "..."
            md_table += f"| {text} | {row['word_count']} |\n"
    
    # Print summary stats in markdown table
    print("Entropy Means by Language:")
    entropy_table = '| Language | Mean | Std Dev | N |\n'
    entropy_table += '|---|---|---|---|\n'
    for lang, group in df_filtered.groupby('language'):
        entropy_table += f"| {lang} | Mean = {group['entropy'].mean():.2f} | Std = {group['entropy'].std():.2f}| N = {len(group)}|\n" 
    print(entropy_table)

    # Check normality (Shapiro-Wilk test) and print in markdown table
    print("\nNormality Test (Shapiro-Wilk):")
    normality_table = '| Language | p-value |\n'
    normality_table += '|--|--|\n'
    for lang, entropy_group in zip(valid_languages, grouped_entropies):
        if len(entropy_group) > 3:  # Shapiro needs > 3 samples
            stat, p = stats.shapiro(entropy_group)
            normality_table += f"|{lang} | p-value = {p:.4f} {'(not normal)' if p < 0.05 else '(normal)'}|\n"
    print(normality_table)
    
    # Check equal variances (Levene's test)
    stat, p = stats.levene(*grouped_entropies)
    print(f"\nLevene's Test for Equal Variances: p-value = {p:.4f} {'(unequal)' if p < 0.05 else '(equal)'}")
    
    # ANOVA (if assumptions hold)
    if all(stats.shapiro(group)[1] >= 0.05 for group in grouped_entropies if len(group) > 3) and p >= 0.05:
        f_stat, p_anova = stats.f_oneway(*grouped_entropies)
        print(f"\nANOVA: F = {f_stat:.2f}, p-value = {p_anova:.4f}")
        if p_anova < 0.05:
            print("Significant difference detected (ANOVA).")
            # Post-hoc: Tukey's HSD
            tukey = pairwise_tukeyhsd(endog=df_filtered['entropy'], 
                                    groups=df_filtered['language'], 
                                    alpha=0.05)
            print(tukey)
    else:
        # Kruskal-Wallis (non-parametric)
        h_stat, p_kruskal = stats.kruskal(*grouped_entropies)
        print(f"\nKruskal-Wallis: H = {h_stat:.2f}, p-value = {p_kruskal:.4f}")
        if p_kruskal < 0.05:
            print("Significant difference detected (Kruskal-Wallis).")
            # Post-hoc: Pairwise Mann-Whitney U (optional)
            from itertools import combinations
            print("\nPairwise Mann-Whitney U Tests (Bonferroni corrected):")
            pairs = list(combinations(valid_languages, 2))
            alpha_corrected = 0.05 / len(pairs)
            for lang1, lang2 in pairs:
                u_stat, p = stats.mannwhitneyu(
                    df_filtered[df_filtered['language'] == lang1]['entropy'],
                    df_filtered[df_filtered['language'] == lang2]['entropy'],
                    alternative='two-sided'
                )
                print(f"{lang1} vs {lang2}: p-value = {p:.4f} {'(significant)' if p < alpha_corrected else ''}")


#### Reviews per Course
- Are there courses with very few or a ton of course reviews?

In [ ]:
# Number of reviews per course
reviews_per_course = df.groupby('course_id').size()
print("Reviews per Course Stats:")
print(f'| Stat | Value |')
print(reviews_per_course.describe().to_markdown())
print(f'| median | {reviews_per_course.median()} |')

Reviews per Course Stats:
| Stat | Value |
|:------|---------:|
| count |   604    |
| mean  |  2408.21 |
| std   |  4599.04 |
| min   |     3    |
| 25%   |   374.25 |
| 50%   |  1071.5  |
| 75%   |  2408.25 |
| max   | 45218    |
| median | 1071.5 |

In [ ]:
# Rebin the course review data to 1-10, 10-100, 100+
plt.figure(figsize=(10, 6))
sns.histplot(reviews_per_course, bins=10)
plt.title('Histogram of Reviews per Course')
plt.xlabel('Number of Reviews')
plt.ylabel('Log(Frequency)')
plt.grid(True, alpha=0.3)
#plt.xscale('log')
#plt.yscale('log')
plt.show()

Not super meaningful. Let's do the same thing with review count as we did with character count

In [ ]:
def plot_review_count_distribution(original_df, id_col='course_id', percentile_cutoff=95, sample_size=5):

    copy_df = original_df.copy()
    df = copy_df.groupby(id_col).size().reset_index(name='review_count')
    
    # Set review count cutoff to a given percentile for plotting. Defult to 95
    cutoff = np.percentile(df['review_count'], percentile_cutoff)
    print(f"\n{percentile_cutoff}th Percentile Cutoff: {cutoff:.0f} reviews")

    main_range = df['review_count'][df['review_count'] <= cutoff]
    outliers = df[df['review_count'] > cutoff]
    
    # Plot histogram for main range
    plt.figure(figsize=(10, 6))
    sns.histplot(main_range, bins=30)
    plt.title(f'Distribution of Course Review Counts (Up to {cutoff:.0f} Reviews)')
    plt.xlabel('Number of Reviews')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('review_count_hist.png')
    plt.show()

    # Make output df of outliers and print to markdown
    if not outliers.empty:
        
        outlier_sample = outliers[[id_col, 'review_count']].head(sample_size)
        
        md_table = "| Course ID | Review Count |\n"
        md_table += "|-------------|------------|\n"
        for _, row in outlier_sample.iterrows():
            # Truncate long text for display (optional)
            text = row[id_col]
            md_table += f"| {text} | {row['review_count']} |\n"
        
        print("\nSample of Outlier Reviews:")
        print(md_table)


plot_review_count_distribution(df, percentile_cutoff=97.5)

Sample of Outlier Reviews:
| Course ID | Review Count |
|-------------|------------|
| ai-for-everyone | 21624 |
| data-scientists-tools | 17595 |
| deep-neural-network | 17850 |
| excel-essentials | 21248 |
| gcp-fundamentals | 17419 |

#### Finding *Meaningful* Reviews
- We'll use word/chracter counts and distributions to calculate entropy to attempt to tag meaningless entries

## Sentiment Analysis - DistilBERT Fine Tuning

In [ ]:
torch.cuda.is_available()

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")